In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay,accuracy_score
from sklearn.model_selection import GridSearchCV
import re
import math

In [2]:
credit_df = pd.read_csv('credit_score/credit_score_data.csv')

C:\Users\jimta\AppData\Local\Temp\ipykernel_12768\1131813828.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  credit_df = pd.read_csv('credit_score/credit_score_data.csv')


In [3]:
credit_df.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [4]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

## Data Preprocessing

Check if all values in Customer_ID are 'normal'(i.e. starts with 'CUS_' and ends with a number or an alphabetical letter) to make sure the Customer_ID data is fully reliable for future use

In [5]:
unique_cust_id = credit_df['Customer_ID'].unique()

In [6]:
all_normal = True
for ID in unique_cust_id:
    if (not ID.startswith('CUS_')) or (not((ID[-1].isalpha()) or (ID[-1].isdigit()))):
        all_normal = False
        break
all_normal

True

In [7]:
# Function for checking if a value is an outlier given the first and the third quantile
def is_outlier(value, q1, q3):
    # Calculate IQR
    iqr = q3 - q1

    # check if the value is an outlier
    threshold = 1.5
    outlier_check = (value < q1 - threshold * iqr) or (value > q3 + threshold * iqr)
    return outlier_check

In [8]:
# Function for getting the mode value of a customer's records(by Customer_ID) on a column
def get_cust_col_mode_val(data, cust_id, col):
    """
    Parameters: 
    data = dataframe object where the data is stored
    cust_id = id of the customer whose the records we want to look into
    col = name of the column we want to get the mode value from
    """
    return data[data['Customer_ID'] == cust_id][col].mode()[0]

By observing Dtype of Age column being 'object' and the Age data, we found that some data entries in Age have non-numeric characters (e.g., trailing underscore) while some other entries have clearly faulty values (i.e., neagtive values or abnormally large values such as 8425, 6476, etc.)

In [9]:
#Get the number of age values with trailing '_'
underscore_count = 0
for age in credit_df['Age']:
    if age.endswith('_'):
        underscore_count += 1
print(underscore_count)

4939


In [10]:
credit_df['Age'].value_counts(sort=False)

Age
23       2537
-500      886
28_       139
28       2829
34       2707
         ... 
8425        1
6476        1
4808_       1
2263        1
1342        1
Name: count, Length: 1788, dtype: int64

In [11]:
type(credit_df['Age'].copy())

pandas.core.series.Series

In [12]:
# Function for preprocessing data in Age column
def Age_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Age'].copy()
    
    # remove all non-numerical characters
    for i in tmp_pd_series.index:
        tmp_pd_series[i] = re.sub("[^0-9-]", "", tmp_pd_series[i])
    # Convert data type of age data from string(object) to int
    tmp_df_copy['Age'] = tmp_pd_series.astype(int)
    tmp_pd_series = tmp_pd_series.astype(int)
    
    # Calculate the first and the third quantile of Age column
    Q1 = tmp_pd_series.quantile(0.25)
    Q3 = tmp_pd_series.quantile(0.75)
    # Replace negative and abnormally large age values with the mode age value
    # of the records for the corresponding customer (referring to the Customer_ID)
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if (tmp_pd_series[i] < 0) or is_outlier(tmp_pd_series[i], Q1, Q3):
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_age = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Age')
            tmp_pd_series[i] = cust_mode_age
       
    return tmp_pd_series

There are over 7000 samples with Occupation value being '\_\_\_\_\_\_\_'. Assuming there will be at least one unique Occupation value other than '\_\_\_\_\_\_\_' for every customer, we will replace '\_\_\_\_\_\_\_' with the mode Occupation value of each corresponding customer(referring to the Customer_ID) or the Occupation value with the second highest frequency for the customer if '\_\_\_\_\_\_\_' is the mode

In [13]:
credit_df['Occupation'].value_counts()

Occupation
_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64

In [14]:
# Function for preprocessing data in Occupation column
def Occup_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Occupation'].copy()
    
    for i in tmp_pd_series.index:
        if tmp_pd_series[i] == '_______':
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_occup = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Occupation')
            if cust_mode_occup == '_______':
                cust_occup_val_counts = tmp_df_copy[tmp_df_copy['Customer_ID'] == cust_id]['Occupation'].value_counts()
                # if the length of array of index of value_counts() output is greater than 1, 
                # it means there is at least one unique Occupation value other than '_______' for this customer, 
                # we will take the second value in the array 
                # which is the unique value with the second highest frequency for this customer
                if len(cust_occup_val_counts.index) > 1:
                    tmp_pd_series[i] = cust_occup_val_counts.index[1]
            else:
                tmp_pd_series[i] = cust_mode_occup  
    
    return tmp_pd_series

By observing Dtype of Annual_Income column being 'object' and the Annual_Income data, we found that some data entries in Annual_Income have non-numeric character(s) other than the decimal points(e.g., trailing underscore), which makes the data type of the column be string(object). 

In [15]:
credit_df['Annual_Income'].value_counts(ascending=True)[:20]

Annual_Income
14834.98_     1
31280.63_     1
12975.96_     1
45780.86_     1
9295469.0     1
24974.35_     1
19205.485_    1
114082.64_    1
30247.7_      1
107360.1_     1
18207.375_    1
100284.24_    1
21225262.0    1
59703.46_     1
30466.74_     1
7750.735_     1
8693.97_      1
14562.32_     1
20570363.0    1
53143.17_     1
Name: count, dtype: int64

In [16]:
def Annual_Income_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Annual_Income'].copy()
    
    # remove all non-numerical or non-decimal-point characters
    for i in tmp_pd_series.index:
        tmp_pd_series[i] = re.sub("[^0-9.]", "", tmp_pd_series[i])
    
    tmp_pd_series = tmp_pd_series.astype(float).round(decimals=2)
    return tmp_pd_series

By observing the dataframe info above and the data on Monthly_Inhand_Salary column in detail, we can see that the Monthly_Inhand_Salary column is of float data type which means there is no faulty non-numerical characters in this column while this column contains 15002 NaN values. We will replace the NaN values witht the most frequent(i.e., mode value) Monthly_Inhand_Salary value of corresponding customer(referring to Customer_ID)

In [17]:
credit_df['Monthly_Inhand_Salary']

0        1824.843333
1                NaN
2                NaN
3                NaN
4        1824.843333
            ...     
99995    3359.415833
99996    3359.415833
99997    3359.415833
99998    3359.415833
99999    3359.415833
Name: Monthly_Inhand_Salary, Length: 100000, dtype: float64

In [18]:
credit_df['Monthly_Inhand_Salary'].value_counts(dropna=False)

Monthly_Inhand_Salary
NaN            15002
2295.058333       15
6082.187500       15
6769.130000       15
6358.956667       15
               ...  
1087.546445        1
3189.212103        1
5640.117744        1
7727.560450        1
2443.654131        1
Name: count, Length: 13236, dtype: int64

In [19]:
def Month_Salary_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Monthly_Inhand_Salary'].copy()
    
    # replace the NaN values witht the most frequent(i.e., mode value) Monthly_Inhand_Salary value 
    # of corresponding customer(referring to Customer_ID)
    for i in tmp_pd_series.index:
        if np.isnan(tmp_pd_series[i]):
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_sal = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Monthly_Inhand_Salary')
            tmp_pd_series[i] = cust_mode_sal
    
    return tmp_pd_series

Num_Bank_Accounts, Num_Credit_Card and Interest_Rate all do not have faulty non-numerical characters and null values, while all three of them contain faulty numerical values(i.e., negative and/or abnormally large) which should be replaced.

In [20]:
credit_df['Num_Bank_Accounts'].value_counts().sort_index()

Num_Bank_Accounts
-1          21
 0        4328
 1        4490
 2        4304
 3       11950
         ...  
 1786        1
 1789        2
 1793        1
 1794        2
 1798        2
Name: count, Length: 943, dtype: int64

In [21]:
credit_df['Num_Credit_Card'].value_counts().sort_index()

Num_Credit_Card
0          13
1        2132
2        2149
3       13277
4       14030
        ...  
1495        1
1496        2
1497        3
1498        3
1499        2
Name: count, Length: 1179, dtype: int64

In [22]:
credit_df['Interest_Rate'].value_counts().sort_index()

Interest_Rate
1       2683
2       2465
3       2765
4       2589
5       4979
        ... 
5775       1
5776       1
5788       1
5789       1
5797       1
Name: count, Length: 1750, dtype: int64

In [23]:
def num_of_account_card_inter_rate_preprocessing(data):
    tmp_df_copy = data.copy()
    col_names = ['Num_Bank_Accounts','Num_Credit_Card','Interest_Rate']
    preprocessed_series_list = []
    for col in col_names: 
        tmp_pd_series = data[col].copy()
        # Calculate the first and the third quantile of current column
        Q1 = tmp_pd_series.quantile(0.25)
        Q3 = tmp_pd_series.quantile(0.75)
        # Replace negative and abnormally large age values with the mode value
        # of the records for the corresponding customer (referring to the Customer_ID)
        # of the mode value is negative, replace it with 0.
        for i in tmp_pd_series.index:
            #print(tmp_pd_series[i])
            if (tmp_pd_series[i] < 0) or is_outlier(tmp_pd_series[i], Q1, Q3):
                cust_id = tmp_df_copy.loc[i]['Customer_ID']
                cust_mode_val = get_cust_col_mode_val(tmp_df_copy, cust_id, col)
                if cust_mode_val < 0:
                    tmp_pd_series[i] = 0
                else:
                    tmp_pd_series[i] = cust_mode_val
        
        # Append the processed series into the final list
        preprocessed_series_list.append(tmp_pd_series)
         
    
    return preprocessed_series_list

Similar to the Age column, the Num_of_Loan column also have some data entries with non-numeric characters (e.g., trailing underscore) while some other entries have clearly faulty values (i.e., neagtive values or abnormally large values such as 8425, 6476, etc.)

In [24]:
# Function for preprocessing data in Num_of_Loan column
def Num_of_Loan_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Num_of_Loan'].copy()
    
    # remove all non-numerical characters
    for i in tmp_pd_series.index:
        tmp_pd_series[i] = re.sub("[^0-9-]", "", tmp_pd_series[i])
    # Convert data type of Num_of_Loan data from string(object) to int
    tmp_df_copy['Num_of_Loan'] = tmp_pd_series.astype(int)
    tmp_pd_series = tmp_pd_series.astype(int)
    
    # Calculate the first and the third quantile of Num_of_Loan column
    Q1 = tmp_pd_series.quantile(0.25)
    Q3 = tmp_pd_series.quantile(0.75)
    # Replace negative and abnormally large values with the mode Num_of_Loan value
    # of the records for the corresponding customer (referring to the Customer_ID)
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if (tmp_pd_series[i] < 0) or is_outlier(tmp_pd_series[i], Q1, Q3):
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_val = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Num_of_Loan')
            tmp_pd_series[i] = cust_mode_val
       
    return tmp_pd_series

By looping through the Type_of_Loan column, we got that there are in total 9 unique loan types that have appeared in the sample dataset.

In [25]:
all_existed_types = []
for types in credit_df['Type_of_Loan']:
    if (type(types) == float) and np.isnan(types):
        continue
    #print(type(types))
    types = types.replace(' and','')
    types = types.split(', ')
    for t in types:
        if t not in all_existed_types:
            all_existed_types.append(t)

all_existed_types

['Auto Loan',
 'Credit-Builder Loan',
 'Personal Loan',
 'Home Equity Loan',
 'Not Specified',
 'Mortgage Loan',
 'Student Loan',
 'Debt Consolidation Loan',
 'Payday Loan']

For each of the 9 unique loan types, we will create a 0-or-1-value column with the name of the corresponding type. If a sample data includes certain loan type, the value on the corresponding column will be 1, 0 otherwise. There are also some entries on the Type_of_Loan column are null. If the Num_of_Loan value of the same data sample is greater than 0, the sample will be categorized as 'Not Specified' loan type.

In [26]:
def Loan_Types_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Type_of_Loan'].copy()
    
    # Initialize a dictionary for future use to create a new dataframe for the loan types
    loan_type_dict = {}
    for loan_type in all_existed_types:
        loan_type_dict[loan_type] = []
        
    for i in tmp_pd_series.index:
        if (type(tmp_pd_series[i]) == float) and np.isnan(tmp_pd_series[i]):
            #If the Num_of_Loan value of the same data sample is greater than 0, 
            # the sample will be categorized as 'Not Specified' loan type.
            if tmp_df_copy.loc[i]['Num_of_Loan'] > 0:
                # append 0 to all loan types except 'Not Specified'
                for loan_type in [Type for Type in all_existed_types if Type != 'Not Specified']:
                    loan_type_dict[loan_type].append(0)
                # append 1 to 'Not Specified' loan type
                loan_type_dict['Not Specified'].append(1)
            else: # Meaning the number of loan for the sample is 0, hence 0 will be appended to all loan types.
                for loan_type in all_existed_types:
                    loan_type_dict[loan_type].append(0)
            continue
        
        else:
            # Get the unique loan types for current sample data
            sample_types = tmp_pd_series[i].replace(' and','')
            sample_types = set(sample_types.split(', '))
            #sample_types = 
            
            #print(sample_types)
            # For types that are not in sample_types, append 0, otherwise append 1
            for loan_type in [Type for Type in all_existed_types if Type not in sample_types]:
                loan_type_dict[loan_type].append(0)
            for loan_type in sample_types:
                loan_type_dict[loan_type].append(1)
                
    # Create a dataframe with the loan_type_dict dictionary
    #for key in loan_type_dict.keys():
    #    print(key)
    #    print(len(loan_type_dict[key]))
    loan_type_df = pd.DataFrame(loan_type_dict, index=tmp_df_copy.index)
    
    return loan_type_df

There seems to be no faulty data in the Delay_from_due_date, neither non-numeric characters nor faulty value(There is some negative values with absolute values less than 10. Considering a customer could pay a loan before the due date, these negative values should not be replace.) has appeared.

In [27]:
np.sort(credit_df['Delay_from_due_date'].values)

array([-5, -5, -5, ..., 67, 67, 67], dtype=int64)

Similar to the Age column, the Num_of_Delayed_Payment column also have some data entries with non-numeric characters (e.g., trailing underscore) while some other entries have clearly faulty values (i.e., neagtive values or abnormally large values such as 4134, 1530, etc.). In addition, there are also over 7000 NaN values in this column. However, by observing the data, unlike Age and Num_of_Loan columns, we have found that the same customer normally could have different Num_of_Delayed_Payment values across different loan sample records. Hence, we decide to remove all the rows with faulty/NaN values instead of replacing these values. 

In [28]:
credit_df['Num_of_Delayed_Payment'].value_counts(dropna=False)

Num_of_Delayed_Payment
NaN     7002
19      5327
17      5261
16      5173
10      5153
        ... 
848_       1
4134       1
1530       1
1502       1
2047       1
Name: count, Length: 750, dtype: int64

In [29]:
def Num_of_Delayed_Payment_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_df_copy.dropna(subset=['Num_of_Delayed_Payment'], inplace=True)
    tmp_pd_series = tmp_df_copy['Num_of_Delayed_Payment'].copy()
    
    # remove all non-numerical characters
    for i in tmp_pd_series.index:
        tmp_pd_series[i] = re.sub("[^0-9-]", "", tmp_pd_series[i])
    # Convert data type of Num_of_Delayed_Payment data from string(object) to int
    tmp_df_copy['Num_of_Delayed_Payment'] = tmp_pd_series.astype(int)
    tmp_pd_series = tmp_pd_series.astype(int)
    
    # Calculate the first and the third quantile of Num_of_Loan column
    Q1 = tmp_pd_series.quantile(0.25)
    Q3 = tmp_pd_series.quantile(0.75)
    # remove the rows with negative or abnormally large values 
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if (tmp_pd_series[i] < 0) or is_outlier(tmp_pd_series[i], Q1, Q3):
            tmp_df_copy.drop(index=i,inplace=True)
       
    return tmp_df_copy

There is no clearly faulty value appeared in the Changed_Credit_Limit column, while '\_' was used for representing no change on credit limit and some of the values has more that 2 number of decimals. We will replace the '\_' values with 0 to make the data type consistent across the column and round all float values to with 2 number of decimals.

In [30]:
credit_df['Changed_Credit_Limit'].value_counts().sort_index()

Changed_Credit_Limit
-0.009999999999999787       4
-0.01999999999999957        3
-0.020000000000000014       3
-0.029999999999999805       1
-0.030000000000000027       1
                         ... 
9.97                       97
9.98                       63
9.989999999999998           2
9.99                       80
_                        2091
Name: count, Length: 4384, dtype: int64

In [31]:
# Function for preprocessing data in Changed_Credit_Limit column
def Credit_Limit_Change_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Changed_Credit_Limit'].copy()
    
    # remove all non-numerical characters
    for i in tmp_pd_series.index:
        if tmp_pd_series[i] == '_': 
            tmp_pd_series[i] = 0
    # Convert data type of Changed_Credit_Limit data from string(object) to float and round with 2 numbers of decimals
    tmp_df_copy['Changed_Credit_Limit'] = (tmp_pd_series.astype(float)).round(decimals=2)
    tmp_pd_series = (tmp_pd_series.astype(float)).round(decimals=2)

    return tmp_pd_series

The Num_Credit_Inquiries column contains some null values and extremely large faulty values(e.g., value greater than 2000). By observing the data, we have found that the Num_Credit_Inquiries values are typically the same for certain customer across different sample loan records of his. Hence, the null and faulty values will be replaced by with the mode value of the records for the corresponding customer (referring to the Customer_ID).

In [32]:
sum(np.isnan(credit_df['Num_Credit_Inquiries']))

1965

In [33]:
credit_df['Num_Credit_Inquiries'].value_counts().sort_index()

Num_Credit_Inquiries
0.0        6972
1.0        7588
2.0        8028
3.0        8890
4.0       11271
          ...  
2588.0        1
2589.0        2
2592.0        2
2594.0        1
2597.0        1
Name: count, Length: 1223, dtype: int64

In [34]:
# Function for preprocessing data in Num_Credit_Inquiries column
def Num_of_CI_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Num_Credit_Inquiries'].copy()
    
    # Calculate the first and the third quantile of Num_of_Loan column
    Q1 = tmp_pd_series.quantile(0.25)
    Q3 = tmp_pd_series.quantile(0.75)
    # Replace negative and abnormally large values with the mode Num_Credit_Inquiries value
    # of the records for the corresponding customer (referring to the Customer_ID)
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if np.isnan(tmp_pd_series[i]) or is_outlier(tmp_pd_series[i], Q1, Q3):
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_val = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Num_Credit_Inquiries')
            tmp_pd_series[i] = cust_mode_val
    
    tmp_pd_series = tmp_pd_series.astype(int)
    return tmp_pd_series

There are 20195 samples with Credit_Mix value being '\_'. By using the groupby method, we have found that the statistical measures of the '\_' class is the closest to those of the Standard class on Monthly_Inhand_Salary, Interest_Rate, Num_of_Loan, Delay_from_due_date, Num_Credit_Inquiries and Credit_Utilization_Ratio. Hence, we will replace the '\_' values with 'Standard'.

In [35]:
credit_df['Credit_Mix'].value_counts(dropna=False)

Credit_Mix
Standard    36479
Good        24337
_           20195
Bad         18989
Name: count, dtype: int64

In [36]:
credit_df.groupby(by = 'Credit_Mix').describe().transpose().loc[['Monthly_Inhand_Salary','Interest_Rate']]

Credit_Mix                            Bad          Good      Standard  \
Monthly_Inhand_Salary count  16183.000000  20630.000000  31032.000000   
                      mean    2742.260214   5679.796637   3983.779596   
                      std     1843.260506   3812.092188   2853.699743   
                      min      303.645417    440.040880    332.128333   
                      25%     1263.518750   2598.077161   1635.065833   
                      50%     1874.053333   4457.680000   2944.613333   
                      75%     4272.377500   8560.935000   5800.276667   
                      max     7268.263333  15204.633333  12514.426667   
Interest_Rate         count  18989.000000  24337.000000  36479.000000   
                      mean      81.649955     69.909151     72.417665   
                      std      454.138274    493.989698    465.899013   
                      min       15.000000      1.000000      1.000000   
                      25%       20.000000      4.000000      9.000000   
                      50%       25.000000      7.000000     15.000000   
                      75%       30.000000     10.000000     19.000000   
                      max     5752.000000   5789.000000   5797.000000   

Credit_Mix                              _  
Monthly_Inhand_Salary count  17153.000000  
                      mean    4157.831504  
                      std     3175.972050  
                      min      303.645417  
                      25%     1617.173333  
                      50%     3046.255000  
                      75%     5902.578333  
                      max    15204.633333  
Interest_Rate         count  20195.000000  
                      mean      66.999257  
                      std      443.930227  
                      min        1.000000  
                      25%        8.000000  
                      50%       14.000000  
                      75%       20.000000  
                      max     5776.000000

In [37]:
credit_df.groupby(by = 'Credit_Mix').describe().transpose().loc[['Delay_from_due_date']]

Credit_Mix                          Bad          Good      Standard  \
Delay_from_due_date count  18989.000000  24337.000000  36479.000000   
                    mean      38.474485     10.032502     19.360098   
                    std       13.979455      7.115920     10.227193   
                    min       10.000000     -5.000000      0.000000   
                    25%       26.000000      5.000000     12.000000   
                    50%       38.000000      9.000000     19.000000   
                    75%       50.000000     14.000000     26.000000   
                    max       67.000000     35.000000     66.000000   

Credit_Mix                            _  
Delay_from_due_date count  20195.000000  
                    mean      21.088784  
                    std       14.927728  
                    min       -5.000000  
                    25%       10.000000  
                    50%       18.000000  
                    75%       28.000000  
                    max       67.000000

In [38]:
credit_df.groupby(by = 'Credit_Mix').describe().transpose().loc[['Num_Credit_Inquiries','Credit_Utilization_Ratio']]

Credit_Mix                               Bad          Good      Standard  \
Num_Credit_Inquiries     count  18619.000000  23859.000000  35758.000000   
                         mean      32.935174     24.691311     26.928799   
                         std      199.916429    190.240357    190.823028   
                         min        6.000000      0.000000      0.000000   
                         25%        8.000000      1.000000      3.000000   
                         50%       10.000000      3.000000      6.000000   
                         75%       12.000000      4.000000      8.000000   
                         max     2573.000000   2592.000000   2597.000000   
Credit_Utilization_Ratio count  18989.000000  24337.000000  36479.000000   
                         mean      31.652473     32.920118     32.183729   
                         std        4.986192      5.233062      5.055938   
                         min       20.172942     20.985919     20.000000   
                         25%       27.388269     28.675987     27.984419   
                         50%       31.769669     32.963784     32.135569   
                         75%       35.920011     37.134126     36.372654   
                         max       43.048022     50.000000     47.555983   

Credit_Mix                                 _  
Num_Credit_Inquiries     count  19799.000000  
                         mean      28.063943  
                         std      194.380781  
                         min        0.000000  
                         25%        3.000000  
                         50%        6.000000  
                         75%        9.000000  
                         max     2588.000000  
Credit_Utilization_Ratio count  20195.000000  
                         mean      32.298157  
                         std        5.120283  
                         min       20.257073  
                         25%       28.051943  
                         50%       32.378326  
                         75%       36.480644  
                         max       49.254983

In [39]:
credit_df['Credit_Mix'].replace(to_replace = '_', value='Standard', inplace=True)

In [40]:
credit_df['Credit_Mix'].value_counts()

Credit_Mix
Standard    56674
Good        24337
Bad         18989
Name: count, dtype: int64

Some of the entries on the Outstanding_Debt contain non-numerical character(s) (e.g., trailing underscore) which need to be removed.

In [41]:
credit_df['Outstanding_Debt'].value_counts()

Outstanding_Debt
1360.45     24
460.46      23
1151.7      23
1109.03     23
467.7       16
            ..
245.46_      1
645.77_      1
174.79_      1
1181.13_     1
1013.53_     1
Name: count, Length: 13178, dtype: int64

In [42]:
def Outstand_Debt_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = tmp_df_copy['Outstanding_Debt'].copy()
    
    # remove all non-numerical characters
    for i in tmp_pd_series.index:
        tmp_pd_series[i] = re.sub("[^0-9.]", "", tmp_pd_series[i])
    # Convert data type of Outstanding_Debt data from string(object) to float
    tmp_pd_series = tmp_pd_series.astype(float)
    
    return tmp_pd_series

There is not any clear faulty data in the Credit_Utilization_Ratio column. Hence, the column does not need to be cleaned and preprocessed.

In [43]:
credit_df['Credit_Utilization_Ratio']

0        26.822620
1        31.944960
2        28.609352
3        31.377862
4        24.797347
           ...    
99995    34.663572
99996    40.565631
99997    41.255522
99998    33.638208
99999    34.192463
Name: Credit_Utilization_Ratio, Length: 100000, dtype: float64

In [44]:
sum(np.isnan(credit_df['Credit_Utilization_Ratio']))

0

In [45]:
print(credit_df['Credit_Utilization_Ratio'].max())
print(credit_df['Credit_Utilization_Ratio'].min())

50.00000000000001
20.0


The Credit_History_Age column contain the length of credit history of customers detailed into the unit of months with string data type. As several months more or less in the credit history would not make a big difference for a customer's credit score. We will round the length of credit history to the unit of years and covert the column into data type of integer. 

There are also some entries are NaN on the Credit_History_Age column. Since by observing the data, we saw the Credit_History_Age values are usually in sequence for the same customer, the NaN values will be replaced by the Credit_History_Age value on the row above it(or loop through the rows below it until a row with the same Customer_ID and not-NaN if NaN Credit_History_Age value is found, if it takes place at the first record for the customer). 

In [46]:
credit_df['Credit_History_Age']

0         22 Years and 1 Months
1                           NaN
2         22 Years and 3 Months
3         22 Years and 4 Months
4         22 Years and 5 Months
                  ...          
99995     31 Years and 6 Months
99996     31 Years and 7 Months
99997     31 Years and 8 Months
99998     31 Years and 9 Months
99999    31 Years and 10 Months
Name: Credit_History_Age, Length: 100000, dtype: object

In [47]:
def Credit_Hist_Age_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = tmp_df_copy['Credit_History_Age'].copy()
    
    # loop through the column first time to conver the string values to float in unit of years
    for i in tmp_pd_series.index:
        if (type(tmp_pd_series[i]) == float) and np.isnan(tmp_pd_series[i]):
            continue
        else:
            months = float(tmp_pd_series[i][tmp_pd_series[i].index('and') + 4])
            years = float(tmp_pd_series[i][:tmp_pd_series[i].index(' Years')])
            # If it is greater than 6 months, round the value 1 year up.
            if months > 6:
                years += 1
            tmp_pd_series[i] = years
    
    # Convert the column from object(string) to float
    tmp_pd_series = tmp_pd_series.astype(float)
    tmp_df_copy['Credit_History_Age'] = tmp_pd_series.astype(float)
    
    # loop through the column second time to replace the NaN values 
    # with the value of the same customer(by Customer_ID) on 1 row above 
    # or loop through the rows below it until a row with the same Customer_ID and not-NaN if NaN Credit_History_Age value is found
    series_index = tmp_pd_series.index
    for i in range(len(series_index)):
        #print(np.isnan(tmp_pd_series[series_index[i]]))
        if np.isnan(tmp_pd_series[series_index[i]]):
            if (tmp_df_copy.loc[series_index[i-1]]['Customer_ID'] 
                == tmp_df_copy.loc[series_index[i]]['Customer_ID']):
                tmp_pd_series[series_index[i]] = tmp_df_copy.loc[series_index[i-1]]['Credit_History_Age']
            else:
                for j in series_index[i:]:
                    if(tmp_df_copy.loc[j]['Customer_ID'] == tmp_df_copy.loc[j]['Customer_ID']):
                        if not np.isnan(tmp_df_copy.loc[j]['Credit_History_Age']):
                            tmp_pd_series[series_index[i]] = tmp_df_copy.loc[j]['Credit_History_Age']
                            break
                        else:
                            continue
                    else:
                        break
    # Conver the data type to int
    # In case there is still some NaN value left, we use dropna
    tmp_df_copy['Credit_History_Age'] = tmp_pd_series
    tmp_pd_series.dropna(inplace=True)
    tmp_df_copy.dropna(subset=['Credit_History_Age'], inplace=True)
    
    #print(tmp_pd_series.value_counts(dropna=False))
    tmp_df_copy['Credit_History_Age'] = tmp_pd_series.astype(int)
    
    return tmp_df_copy
    
            

The Payment_of_Min_Amount column has 3 unique categories. There is not any null or faulty values. We are not sure what the 'NM' exactly means(probably 'Not Mandatory'?). But because the statistical measures for the samples of the 'NM' category is not really close to that of either one of other two categories, we decide to leave the column as it is.

In [48]:
credit_df['Payment_of_Min_Amount'].value_counts()

Payment_of_Min_Amount
Yes    52326
No     35667
NM     12007
Name: count, dtype: int64

In [49]:
credit_df.groupby(by = 'Payment_of_Min_Amount').describe().transpose().loc[['Monthly_Inhand_Salary','Interest_Rate']]

Payment_of_Min_Amount                  NM            No           Yes
Monthly_Inhand_Salary count  10170.000000  30228.000000  44600.000000
                      mean    4221.183928   5379.845808   3384.410633
                      std     3215.221088   3662.817491   2501.660143
                      min      332.431250    440.040880    303.645417
                      25%     1630.684167   2438.357500   1417.340833
                      50%     3104.813333   3828.355000   2644.078333
                      75%     5964.883333   8043.110000   4940.072500
                      max    15167.180000  15204.633333  12514.426667
Interest_Rate         count  12007.000000  35667.000000  52326.000000
                      mean      72.538769     67.754507     75.660876
                      std      465.430319    480.876629    456.521871
                      min        1.000000      1.000000      5.000000
                      25%        8.000000      4.000000     14.000000
                      50%       14.000000      8.000000     19.000000
                      75%       20.000000     11.000000     26.000000
                      max     5774.000000   5789.000000   5797.000000

In [50]:
credit_df.groupby(by = 'Payment_of_Min_Amount').describe().transpose().loc[['Delay_from_due_date']]

Payment_of_Min_Amount                NM            No           Yes
Delay_from_due_date count  12007.000000  35667.000000  52326.000000
                    mean      21.120596     11.898982     27.307304
                    std       14.831379      7.996447     15.222453
                    min       -5.000000     -5.000000      0.000000
                    25%       10.000000      6.000000     16.000000
                    50%       18.000000     11.000000     24.000000
                    75%       28.000000     16.000000     36.000000
                    max       67.000000     35.000000     67.000000

In [51]:
credit_df.groupby(by = 'Payment_of_Min_Amount').describe().transpose().loc[['Num_Credit_Inquiries','Credit_Utilization_Ratio']]

Payment_of_Min_Amount                     NM            No           Yes
Num_Credit_Inquiries     count  11791.000000  34939.000000  51305.000000
                         mean      26.448732     24.540313     30.242998
                         std      185.137721    190.559122    196.693877
                         min        0.000000      0.000000      0.000000
                         25%        3.000000      1.000000      6.000000
                         50%        6.000000      3.000000      8.000000
                         75%        9.000000      4.000000     11.000000
                         max     2592.000000   2592.000000   2597.000000
Credit_Utilization_Ratio count  12007.000000  35667.000000  52326.000000
                         mean      32.247081     32.807988     31.937546
                         std        5.100680      5.217457      5.020495
                         min       20.985606     20.985919     20.000000
                         25%       28.073692     28.543177     27.730882
                         50%       32.301163     32.865462     31.963726
                         75%       36.415571     37.030270     36.136860
                         max       49.254983     50.000000     47.555983

The Total_EMI_per_month column contains some extremely large faulty values(e.g., value greater than 10000). By observing the data, we have found that the Total_EMI_per_month values are typically the same for certain customer across different sample loan records of his. Hence, the faulty values will be replaced by with the mode value of the records for the corresponding customer (referring to the Customer_ID).

In [52]:
credit_df['Total_EMI_per_month'].value_counts(dropna=False)

Total_EMI_per_month
0.000000        10613
49.574949           8
73.533361           8
22.960835           8
38.661127           8
                ...  
36408.000000        1
23760.000000        1
24612.000000        1
24325.000000        1
58638.000000        1
Name: count, Length: 14950, dtype: int64

In [53]:
# Function for preprocessing data in Total_EMI_per_month column
def EMI_per_Month_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_pd_series = data['Total_EMI_per_month'].copy()
    
    # Calculate the first and the third quantile of Num_of_Loan column
    Q1 = tmp_pd_series.quantile(0.25)
    Q3 = tmp_pd_series.quantile(0.75)
    # Replace abnormally large values with the mode Num_of_Loan value
    # of the records for the corresponding customer (referring to the Customer_ID)
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if is_outlier(tmp_pd_series[i], Q1, Q3):
            cust_id = tmp_df_copy.loc[i]['Customer_ID']
            cust_mode_val = get_cust_col_mode_val(tmp_df_copy, cust_id, 'Total_EMI_per_month')
            tmp_pd_series[i] = cust_mode_val
    
    tmp_pd_series = tmp_pd_series.round(decimals=2)
    return tmp_pd_series

In the Amount_invested_monthly column there are more than 4000 rows of NaN values and \_\_10000\_\_ respectively. Because the Amount_invested_monthly values of the same customer vary across different sample records and we don't know what the \_\_10000\_\_ means, we decided to drop the sample data with NaN and \_\_10000\_\_ values.

In [54]:
credit_df['Amount_invested_monthly'].value_counts(dropna=False)

Amount_invested_monthly
NaN                   4479
__10000__             4305
0.0                    169
36.66235139442514        1
89.7384893604547         1
                      ... 
36.541908593249026       1
93.45116318631192        1
140.80972223052834       1
38.73937670100975        1
167.1638651610451        1
Name: count, Length: 91050, dtype: int64

In [55]:
def Month_Invest_Amount_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_df_copy.dropna(subset=['Amount_invested_monthly'], inplace=True)
    tmp_pd_series = tmp_df_copy['Amount_invested_monthly'].copy()
    
    # remove the rows with '__10000__' Amount_invested_monthly value
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if tmp_pd_series[i] == '__10000__':
            tmp_df_copy.drop(index=i,inplace=True)
            
    # Convert the Amount_invested_monthly column to float and round to with 2 numbers of decimals
    tmp_df_copy['Amount_invested_monthly'] = tmp_df_copy['Amount_invested_monthly'].astype(float).round(decimals=2)
    return tmp_df_copy

In the Payment_Behaviour column there are 7600 rows with '!@9#%8' value. Because the Payment_Behaviour values of the same customer vary across different sample records and we don't know what '!@9#%8' means, we decided to drop the sample data with '!@9#%8' value.

In addition we will split the Payment_Behaviour into three 0-or-1 valued columns namely 'Low\_Spent', 'Small\_value\_payments' and 'Medium\_value\_payments' respectively.

In [56]:
credit_df['Payment_Behaviour'].value_counts(dropna=False)

Payment_Behaviour
Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: count, dtype: int64

In [4]:
def Payment_Behaviour_preprocessing(data):
    tmp_df_copy = data.copy()
    
    # Initialize a dictionary for future use to create a new dataframe for the loan types
    behaviour_types = ['Low_Spent', 'Small_value_payments', 'Medium_value_payments']
    behaviour_types = ['Payment_Behaviour_'+Type for Type in behaviour_types]
    behav_type_dict = {}
    for behav_type in behaviour_types:
        behav_type_dict[behav_type] = []
    
    # Drop the rows with '!@9#%8' Payment_Behaviour value in the first loop
    for i in tmp_df_copy.index:
        if tmp_df_copy.loc[i]['Payment_Behaviour'] == '!@9#%8':
            tmp_df_copy.drop(index=i,inplace=True)
    
    
    tmp_pd_series = tmp_df_copy['Payment_Behaviour'].copy()
    for i in tmp_pd_series.index:
        # Get the unique loan types for current sample data
        pay_behav_type = tmp_pd_series[i].split('_spent_')
        spent_type = pay_behav_type[0]
        value_payment_type = pay_behav_type[1].replace('_value_payments','')
            
        #print(spent_type)
        #print(value_payment_type)
        # Add 0 or 1 into the corresponding lists in behav_type_dict
        if spent_type == 'Low':
            behav_type_dict['Payment_Behaviour_Low_Spent'].append(1)
        else:
            behav_type_dict['Payment_Behaviour_Low_Spent'].append(0)
        if value_payment_type == 'Small':
            behav_type_dict['Payment_Behaviour_Small_value_payments'].append(1)
            behav_type_dict['Payment_Behaviour_Medium_value_payments'].append(0)
        elif value_payment_type == 'Medium':
            behav_type_dict['Payment_Behaviour_Small_value_payments'].append(0)
            behav_type_dict['Payment_Behaviour_Medium_value_payments'].append(1)
        else:
            behav_type_dict['Payment_Behaviour_Small_value_payments'].append(0)
            behav_type_dict['Payment_Behaviour_Medium_value_payments'].append(0)
        
                
    # Create a dataframe with the behav_type_dict dictionary
    behav_type_df = pd.DataFrame(behav_type_dict, index=tmp_df_copy.index)
    # Concatenate the behav_type_df with the tmp_df_copy and drop the original Payment_Behaviour column
    tmp_df_copy = pd.concat([tmp_df_copy.drop(columns=['Payment_Behaviour']),behav_type_df], axis = 1)
    
    return tmp_df_copy

In the Monthly_Balance column there are 1200 and 9 rows of NaN values and '\_\_-333333333333333333333333333\_\_' respectively. Because the Monthly_Balance values of the same customer vary across different sample records and we don't know what the '\_\_-333333333333333333333333333\_\_' means, we decided to drop the sample data with NaN and '\_\_-333333333333333333333333333\_\_' values. 

In addition, lots of the values in Monthly_Balance have more than 2 numbers of decimals which does not make sense. We will round the values down to with only 2 numbers of decimals.

In [58]:
credit_df['Monthly_Balance'].value_counts(dropna=False)

Monthly_Balance
NaN                                 1200
__-333333333333333333333333333__       9
312.49408867943663                     1
347.413889978152                       1
254.9709216273975                      1
                                    ... 
366.2890379762706                      1
151.1882696261166                      1
306.75027851710234                     1
278.8720257394474                      1
393.6736955618808                      1
Name: count, Length: 98793, dtype: int64

In [59]:
def Month_Balance_preprocessing(data):
    tmp_df_copy = data.copy()
    tmp_df_copy.dropna(subset=['Monthly_Balance'], inplace=True)
    tmp_pd_series = tmp_df_copy['Monthly_Balance'].copy()
    
    # remove the rows with '__-333333333333333333333333333__' Monthly_Balance value
    for i in tmp_pd_series.index:
        #print(tmp_pd_series[i])
        if tmp_pd_series[i] == '__-333333333333333333333333333__':
            tmp_df_copy.drop(index=i,inplace=True)
            
    # Convert the Monthly_Balance column to float and round to with 2 numbers of decimals
    tmp_df_copy['Monthly_Balance'] = tmp_df_copy['Monthly_Balance'].astype(float).round(decimals=2)
    return tmp_df_copy

## Run all the preprocessing functions defined above to preprocess the data 

In [60]:
credit_df['Age'] = Age_preprocessing(credit_df)
credit_df['Age'].describe()

count    100000.000000
mean         33.311180
std          10.764783
min          14.000000
25%          24.000000
50%          33.000000
75%          42.000000
max          56.000000
Name: Age, dtype: float64

In [61]:
credit_df['Occupation'] = Occup_preprocessing(credit_df)
credit_df['Occupation'].value_counts()

Occupation
Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6720
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: count, dtype: int64

In [62]:
credit_df['Annual_Income'] = Annual_Income_preprocessing(credit_df)
credit_df['Annual_Income'].describe()

count    1.000000e+05
mean     1.764157e+05
std      1.429618e+06
min      7.005930e+03
25%      1.945750e+04
50%      3.757861e+04
75%      7.279092e+04
max      2.419806e+07
Name: Annual_Income, dtype: float64

In [63]:
credit_df['Monthly_Inhand_Salary'] = Month_Salary_preprocessing(credit_df)
credit_df['Monthly_Inhand_Salary'].describe()

count    100000.000000
mean       4198.350924
std        3187.402015
min         303.645417
25%        1626.761667
50%        3095.978333
75%        5961.637500
max       15204.633333
Name: Monthly_Inhand_Salary, dtype: float64

In [64]:
credit_df['Num_Bank_Accounts'], credit_df['Num_Credit_Card'], credit_df['Interest_Rate'] = num_of_account_card_inter_rate_preprocessing(credit_df)

In [65]:
credit_df['Num_Bank_Accounts'].describe()

count    100000.000000
mean          5.368840
std           2.593273
min           0.000000
25%           3.000000
50%           5.000000
75%           7.000000
max          11.000000
Name: Num_Bank_Accounts, dtype: float64

In [66]:
credit_df['Num_Credit_Card'].describe()

count    100000.000000
mean          5.533680
std           2.067126
min           0.000000
25%           4.000000
50%           5.000000
75%           7.000000
max          11.000000
Name: Num_Credit_Card, dtype: float64

In [67]:
credit_df['Interest_Rate'].describe()

count    100000.00000
mean         14.53208
std           8.74133
min           1.00000
25%           7.00000
50%          13.00000
75%          20.00000
max          34.00000
Name: Interest_Rate, dtype: float64

In [68]:
credit_df['Num_of_Loan'] = Num_of_Loan_preprocessing(credit_df)

In [69]:
credit_df['Num_of_Loan'].describe()

count    100000.000000
mean          3.532880
std           2.446356
min           0.000000
25%           2.000000
50%           3.000000
75%           5.000000
max           9.000000
Name: Num_of_Loan, dtype: float64

In [70]:
credit_df = pd.concat([credit_df.drop(columns=['Type_of_Loan']),Loan_Types_preprocessing(credit_df)], axis = 1)

In [71]:
credit_df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Score,Auto Loan,Credit-Builder Loan,Personal Loan,Home Equity Loan,Not Specified,Mortgage Loan,Student Loan,Debt Consolidation Loan,Payday Loan
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,1,1,0,0,0,0,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,1,1,0,0,0,0,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,1,1,0,0,0,0,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,1,1,0,0,0,0,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Poor,1,0,0,0,0,0,1,0,0
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Poor,1,0,0,0,0,0,1,0,0
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Poor,1,0,0,0,0,0,1,0,0
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Standard,1,0,0,0,0,0,1,0,0


In [72]:
credit_df = Num_of_Delayed_Payment_preprocessing(credit_df)

In [73]:
credit_df['Num_of_Delayed_Payment'].describe()

count    91618.000000
mean        13.419634
std          6.207354
min          0.000000
25%          9.000000
50%         14.000000
75%         18.000000
max         28.000000
Name: Num_of_Delayed_Payment, dtype: float64

In [74]:
credit_df['Changed_Credit_Limit'] = Credit_Limit_Change_preprocessing(credit_df)

In [75]:
credit_df['Changed_Credit_Limit'].describe()

count    91618.000000
mean        10.196895
std          6.884127
min         -6.490000
25%          4.980000
50%          9.280000
75%         14.730000
max         36.970000
Name: Changed_Credit_Limit, dtype: float64

In [76]:
credit_df['Num_Credit_Inquiries'] = Num_of_CI_preprocessing(credit_df)

In [77]:
credit_df['Num_Credit_Inquiries'].describe()

count    91618.000000
mean         5.805049
std          3.860227
min          0.000000
25%          3.000000
50%          5.000000
75%          9.000000
max         17.000000
Name: Num_Credit_Inquiries, dtype: float64

In [78]:
credit_df['Outstanding_Debt'] = Outstand_Debt_preprocessing(credit_df)

In [79]:
credit_df['Outstanding_Debt'].describe()

count    91618.000000
mean      1430.803249
std       1156.583480
min          0.230000
25%        568.590000
50%       1169.610000
75%       1960.280000
max       4998.070000
Name: Outstanding_Debt, dtype: float64

In [80]:
credit_df = Credit_Hist_Age_preprocessing(credit_df)

In [81]:
credit_df['Credit_History_Age'].value_counts(dropna=False)

Credit_History_Age
18    4856
19    4746
17    4639
16    4602
20    3795
15    3147
13    3048
6     2944
30    2927
32    2853
29    2846
8     2803
31    2792
12    2777
24    2743
9     2734
22    2723
21    2716
11    2705
23    2701
10    2700
14    2694
26    2691
7     2678
28    2673
27    2671
25    2568
33    1743
5     1601
2      980
1      935
3      852
4      755
0      326
34      26
Name: count, dtype: int64

In [82]:
credit_df['Total_EMI_per_month'] = EMI_per_Month_preprocessing(credit_df)

In [83]:
credit_df['Total_EMI_per_month'].describe()

count    90990.000000
mean       104.304882
std        120.122082
min          0.000000
25%         29.180000
50%         66.260000
75%        145.615000
max       1701.960000
Name: Total_EMI_per_month, dtype: float64

In [84]:
credit_df = Month_Invest_Amount_preprocessing(credit_df)

In [85]:
credit_df['Amount_invested_monthly'].value_counts(dropna=False)

Amount_invested_monthly
0.00      154
42.58      13
67.88      13
117.24     12
63.67      12
         ... 
100.09      1
93.73       1
466.36      1
239.77      1
167.16      1
Name: count, Length: 36898, dtype: int64

In [5]:
credit_df = Payment_Behaviour_preprocessing(credit_df)

In [7]:
credit_df[['Payment_Behaviour','Payment_Behaviour_Low_Spent', 'Payment_Behaviour_Small_value_payments', 'Payment_Behaviour_Medium_value_payments']]

,Payment_Behaviour,Payment_Behaviour_Low_Spent,Payment_Behaviour_Small_value_payments,Payment_Behaviour_Medium_value_payments
0,High_spent_Small_value_payments,0,1,0
1,Low_spent_Large_value_payments,1,0,0
2,Low_spent_Medium_value_payments,1,0,1
3,Low_spent_Small_value_payments,1,1,0
4,High_spent_Medium_value_payments,0,0,1
...,...,...,...,...
99994,High_spent_Medium_value_payments,0,0,1
99995,High_spent_Large_value_payments,0,0,0
99996,High_spent_Medium_value_payments,0,0,1
99997,High_spent_Large_value_payments,0,0,0


In [88]:
credit_df = Month_Balance_preprocessing(credit_df)

In [89]:
credit_df['Monthly_Balance'].describe()

count    75821.000000
mean       401.752618
std        213.612704
min          0.010000
25%        269.760000
50%        336.010000
75%        468.620000
max       1576.290000
Name: Monthly_Balance, dtype: float64

In [90]:
credit_df.describe()

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Personal Loan,Home Equity Loan,Not Specified,Mortgage Loan,Student Loan,Debt Consolidation Loan,Payday Loan,Payment_Behaviour_Low_Spent,Payment_Behaviour_Small_value_payments,Payment_Behaviour_Medium_value_payments
count,75821.000000,7.582100e+04,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,...,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000,75821.000000
mean,33.260930,1.781567e+05,4145.134405,5.381293,5.544809,14.577320,3.524208,21.128249,13.423207,10.206444,...,0.311431,0.312684,0.316153,0.312209,0.310771,0.310811,0.319832,0.534179,0.395036,0.342293
std,10.767456,1.448595e+06,3166.134514,2.584578,2.061796,8.752473,2.441132,14.879676,6.207045,6.873881,...,0.463082,0.463590,0.464976,0.463398,0.462812,0.462829,0.466414,0.498834,0.488862,0.474480
min,14.000000,7.005930e+03,303.645417,0.000000,0.000000,1.000000,0.000000,-5.000000,0.000000,-6.490000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24.000000,1.926727e+04,1615.457500,3.000000,4.000000,7.000000,2.000000,10.000000,9.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,3.682229e+04,3045.680000,6.000000,5.000000,13.000000,3.000000,18.000000,14.000000,9.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,42.000000,7.189294e+04,5896.016667,7.000000,7.000000,20.000000,5.000000,28.000000,18.000000,14.740000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,56.000000,2.419806e+07,15204.633333,11.000000,11.000000,34.000000,9.000000,67.000000,28.000000,36.970000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [91]:
credit_df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance', 'Credit_Score',
       'Auto Loan', 'Credit-Builder Loan', 'Personal Loan', 'Home Equity Loan',
       'Not Specified', 'Mortgage Loan', 'Student Loan',
       'Debt Consolidation Loan', 'Payday Loan', 'Payment_Behaviour_Low_Spent',
       'Payment_Behaviour_Small_value_payments',
       'Payment_Behaviour_Medium_value_payments'],
      dtype='object')

In [92]:
print(sum(credit_df.drop(columns=['Month','Name','SSN']).isna().any(axis=1)))

0


In [93]:
credit_df = credit_df.reset_index(drop=True)

In [94]:
credit_df.to_csv('credit_score/credit_score_data_cleaned.csv')